# Clipping plane visualization of tent slab solution

We show how to convert a tent slab based on a 2D mesh into a 3D mesh for which we can use the webgui's clipping feature.

In [1]:
from netgen.geom2d import unit_square
from ngsolve import Mesh, CoefficientFunction, x, y, exp, L2, H1, GridFunction, TaskManager
from ngstents import TentSlab
from ngstents.conslaw import Burgers
from ngsolve.webgui import Draw
from ngstents.utils import SlabConverter

In [2]:
help(SlabConverter)

Help on class SlabConverter in module ngstents.utils.slab2mesh:

class SlabConverter(builtins.object)
 |  SlabConverter(tps, p_hd=1)
 |  
 |  A class for converting a tent slab to a valid NGSolve mesh.
 |  Provides a method 'Convert' to generate the mesh.
 |  Once the conversion has been performed, a number of
 |  data structures are available as attributes of the converter object.
 |  These include 'mesh', which is the generated mesh and 'gfixmap',
 |  used with an ngstents conservation law to view time slices of a solution.
 |  'gfixmap' is a list of dicts, one for each front, mapping a tent vertex
 |  pitched in that front to the corresponding vertex of the generated mesh.
 |  If the constructor is called with p_hd = 2, the tent's internal edges
 |  are also mapped to edges of the generated mesh.
 |  
 |  Methods defined here:
 |  
 |  Convert(self, tscale=1.0)
 |      Convert the tent pitched slab to an N-D mesh, providing
 |      timing information and counts.
 |  
 |  __init__(se

In [7]:
maxh = 0.05
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))

heapsize = 10*1000*1000
dt = 0.05
c = 6
ts = TentSlab(mesh, heapsize=heapsize)
ts.SetMaxWavespeed(c)
ts.PitchTents(dt)
print("max slope", ts.MaxSlope())

order = 4
V = L2(mesh, order=order)
u = GridFunction(V,"u")

burg = Burgers(u, ts, outflow=mesh.Boundaries("left|bottom|right|top"))
burg.SetTentSolver("SARK",substeps=order*order)

cf = CoefficientFunction(exp(-50*((x-0.3)*(x-0.3)+(y-0.3)*(y-0.3))))
burg.SetInitial(cf)

scene = Draw(u)

# SlabConverter can be used to convert a tent slab based on a 2D spatial mesh to a 3D mesh.
# order_3d must be 1 or 2
order_3d = 2
sc = SlabConverter(ts, p_hd=order_3d)
sc.Convert(tscale=5)
mesh3d = sc.mesh
gfixmap = sc.gfixmap

# Due to memory concerns, we currently store only interpolated vertex dofs
# (and optionally edge dofs if order_3d=2) for the 3D space.
# We define an H1 order 1 or 2 space and associated GridFunction here.
V3d = H1(sc.mesh, order=order_3d)
u3d = GridFunction(V3d, "u3d")

# Set the nested list which gives 3D mesh vertex number
# for front and spatial vertex number
burg.SetIdx3d(gfixmap)

tend = 10*dt

# Define the time steps for which we want to examine the solution using clipping
vecs = {1: u3d.vec.CreateVector(), 10: u3d.vec.CreateVector()}

t = 0
n = 1

with TaskManager():
    while t < tend - dt/2:
        if n in vecs:
            # Tell Propagate to update the provided vector with the average
            # value for each tent vertex after propagating on the tent.
            u3d.vec.data = vecs[n]
            burg.Propagate(hdgf=u3d)  # Propagate updates u3d 
            vecs[n].data = u3d.vec    # store updated solution vector
        else:
            burg.Propagate()
        t += dt
        n += 1
        print("{:.3f}".format(t))
        scene.Redraw()

max slope 0.23813840207188286


WebGuiWidget(value={'ngsolve_version': '6.2.2102-124-gbb6f627c8', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

add vertices 0.07698
add volume elements 0.40785
add surface elements 0.28961
make ngsolve mesh 0.04575
make index map 0.6988534927368164
3623 verts, 17104 vol elems, 3958 surf elems in 1.51993.
0.050
0.100
0.150
0.200
0.250
0.300
0.350
0.400
0.450
0.500


## Now we can also visualize the solution on a 3D mesh using clipping

In [8]:
u3d.vec.data = vecs[1]
clip = {'vec': [0, 0, -1], 'dist': 0.01} 
Draw(u3d, clipping=clip)

WebGuiWidget(value={'ngsolve_version': '6.2.2102-124-gbb6f627c8', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, '…

BaseWebGuiScene

In [9]:
u3d.vec.data = vecs[10]
clip = {'vec': [0, 0, -1], 'dist': 0.01} 
Draw(u3d, clipping=clip)


WebGuiWidget(value={'ngsolve_version': '6.2.2102-124-gbb6f627c8', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, '…

BaseWebGuiScene

## Observations
We can visualize the solution at an arbitrary time, but the H1 order 1 approximation is not very nice compared to the L2 order 4 solution.

Try changing the H1 order to 2 to see an improvement!